In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from bench_lib.evaluation import compute_agreement_score, load_human_labels
from bench_lib.utils import enable_info_logs
import pandas as pd

enable_info_logs()

human_labels, questions, comment_cols = load_human_labels()
human_labels_long = pd.melt(
    human_labels, id_vars=["post_id", "classification_by"], value_vars=questions
)
human_consistency_df = compute_agreement_score(human_labels_long)
human_consistency_df.head(2)

from bench_lib.evaluation import benchmark_results_folder, check_self_consistency


folder = "self-consistency"
csv = benchmark_results_folder() / folder / "model_labels.csv"
flip_counts_all, consistency_score = check_self_consistency(
    csv_paths=[str(csv)],
    model_to_check="google/gemma-3-27b-it",
    n_runs=5,
)

In [ ]:
from bench_lib.evaluation import load_ai_labels

folder = "self-consistency"
ai_labels_long = load_ai_labels([folder], questions, comment_cols)
ai_labels_long.head(2)

In [ ]:
from bench_lib.evaluation import agreement_score


ai_agreement_df = compute_agreement_score(
    ai_labels_long, groupby=["post_id", "Model ID", "variable"]
)
ai_agreement_df.head(2)

In [ ]:
import pandas as pd

joint_consistency = pd.merge(
    ai_agreement_df,
    human_consistency_df,
    on=["post_id", "variable"],
    how="left",
    suffixes=("_ai", "_human"),
)
joint_consistency.head(2)

In [ ]:
large_model = ["google/gemma-3-27b-it", "Qwen/Qwen2.5-VL-72B-Instruct"]
joint_consistency.query("`Model ID` in @large_model").groupby(["Model ID"]).agg(
    agreement_score_human=("agreement_score_human", "mean"),
    agreement_score_ai=("agreement_score_ai", "mean"),
)

# Plotting

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns


def plot_human_ai_agreement_corr(**lm_plot_kwargs) -> plt.Figure:
    large_model = ["google/gemma-3-27b-it", "Qwen/Qwen2.5-VL-72B-Instruct"]
    fig = sns.lmplot(
        data=joint_consistency.query("`Model ID` in @large_model"),
        x="agreement_score_human",
        y="agreement_score_ai",
        y_jitter=0.02,
        x_jitter=0.02,
        **lm_plot_kwargs,
    )
    fig.set_ylabels("AI Agreement Score")
    fig.set_xlabels("Human Agreement Score")
    for ax in fig.axes.flat:
        ax.grid(alpha=0.5)
        ax.set_ylim(-0.05, 1.05)
    fig
    return fig


fig = plot_human_ai_agreement_corr()
fig.savefig("imgs/ai_consistency_by_human_consistency.pdf", bbox_inches="tight")
fig2 = plot_human_ai_agreement_corr(hue="variable", row="Model ID", col="variable")
fig2.savefig("imgs/ai_consistency_by_human_consistency_disagg.pdf", bbox_inches="tight")